In [2]:
import gct

In [3]:
prefix = "B_alg_runtime_vs_graph_size"

def setup_data(N):
    #gct.remove_data(prefix + '_*', dry_run=False)
    graph_unweighted_undirect = gct.random_dataset.generate_undirected_unweighted_random_graph_LFR(name=prefix + "_N"+str(N), \
                                       N=N, k=int(N/4), maxk=int(N/2), mu=0.2, minc=int(N/10))
    data1 = graph_unweighted_undirect
    return data1


In [4]:
%time data=setup_data(4096)
data.name

2020-04-24 00:24:34,155 - RandomGenerator:LFR - INFO - Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_undir_net -minc 409 -mu 0.2 -maxk 2048 -k 1024 -N 4096'  with seed 469630
INFO:RandomGenerator:LFR:Runing '/opt/gct/submodules/CommunityDetectionCodes/lfr_undir_net -minc 409 -mu 0.2 -maxk 2048 -k 1024 -N 4096'  with seed 469630
2020-04-24 00:24:34,156 - RandomGenerator:LFR - INFO - working dir: /tmp/tmp4fhm7sfz
INFO:RandomGenerator:LFR:working dir: /tmp/tmp4fhm7sfz
2020-04-24 01:03:05,623 - RandomGenerator:LFR - INFO - min node is 1. will make it 0
INFO:RandomGenerator:LFR:min node is 1. will make it 0
/opt/conda/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
2020-04-24 01:03:06,493 - Dataset:B_a

CPU times: user 1.53 s, sys: 351 ms, total: 1.88 s
Wall time: 38min 32s


'B_alg_runtime_vs_graph_size_N4096'

In [5]:
!pip install func_timeout

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import os
from gct import config,utils
from func_timeout import func_timeout, FunctionTimedOut
os.environ['OMP_NUM_THREADS']="1"
def has_run(runame, dsname):
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', runame,dsname))
    return utils.file_exists(fpath)

def run_alg(alg,dsname, check_skip=True):
    try:
        runname = alg
        if not check_skip or (check_skip and not has_run(runname, dsname)):
            print ("runing ", alg, dsname)
            if alg =='cdc_SVINET':
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123,  max_iterations=1000)        
            else:
                gct.run_alg(runname=runname, data=gct.load_local_graph(dsname), algname=alg, seed=123)
            
            print ("finished", alg,dsname)
        else:
            print("skip", runname,dsname)
    except:
        print("except", runname,dsname)
    return True

def timeout(func, args=(), kwargs={}, timeout_duration=10, default=None):
    result=default
    try:
        result = func_timeout(timeout_duration, func, args=args, kwargs=kwargs)
    except FunctionTimedOut:
        print("timeout", func,args,kwargs)
    except:
        print ("Unexpected error:", sys.exc_info()[0])
    return result


def run_alg_timeout(alg,dsname,timeout_duration=600):
    if not timeout(run_alg, (alg,dsname), timeout_duration=timeout_duration):
        print("tiemout", alg,dsname)
        return False
    fpath = os.path.join("/{}/tmp/{}_{}.done".format('data', alg,dsname))
    #!touch {fpath}
    open(fpath, 'a').close()
    return True

        
def run_alg_timeout_mp(ab):
    return run_alg_timeout(ab[0],ab[1],3600)

In [7]:
run_alg_timeout("alg_pg_label_propagation", data.name)

2020-04-24 01:03:07,625 - alg_pg_label_propagation - INFO - seed ignored
INFO:alg_pg_label_propagation:seed ignored
2020-04-24 01:03:07,625 - Dataset:B_alg_runtime_vs_graph_size_N4096 - INFO - writing edges to /data/data/B_alg_runtime_vs_graph_size_N4096/edges.txt
INFO:Dataset:B_alg_runtime_vs_graph_size_N4096:writing edges to /data/data/B_alg_runtime_vs_graph_size_N4096/edges.txt
2020-04-24 01:03:07,627 - Dataset:B_alg_runtime_vs_graph_size_N4096 - INFO - reading /data/data/B_alg_runtime_vs_graph_size_N4096/edges.parq
INFO:Dataset:B_alg_runtime_vs_graph_size_N4096:reading /data/data/B_alg_runtime_vs_graph_size_N4096/edges.parq


runing  alg_pg_label_propagation B_alg_runtime_vs_graph_size_N4096


2020-04-24 01:07:02,509 - Dataset:B_alg_runtime_vs_graph_size_N4096 - INFO - finish writing /data/data/B_alg_runtime_vs_graph_size_N4096/edges.txt
INFO:Dataset:B_alg_runtime_vs_graph_size_N4096:finish writing /data/data/B_alg_runtime_vs_graph_size_N4096/edges.txt
2020-04-24 01:07:02,510 - alg_pg_label_propagation - INFO - Running /opt/gct/submodules/PowerGraph/label_propagation --graph /tmp/tmp90q06n3g/edges.txt --saveprefix=output.cluster --execution async --ncpus 1 --weighted 0 --directed 0
INFO:alg_pg_label_propagation:Running /opt/gct/submodules/PowerGraph/label_propagation --graph /tmp/tmp90q06n3g/edges.txt --saveprefix=output.cluster --execution async --ncpus 1 --weighted 0 --directed 0
2020-04-24 01:07:07,092 - alg_pg_label_propagation - INFO - Made 2 clusters in 4.568308 seconds
INFO:alg_pg_label_propagation:Made 2 clusters in 4.568308 seconds


finished alg_pg_label_propagation B_alg_runtime_vs_graph_size_N4096


True

In [8]:
run_alg_timeout("alg_GossipMap", data.name)

2020-04-24 01:07:07,103 - alg_GossipMap - INFO - seed ignored
INFO:alg_GossipMap:seed ignored
2020-04-24 01:07:07,106 - alg_GossipMap - INFO - Running /opt/gct/submodules/PowerGraph/GossipMap --graph /tmp/tmpes78bgty/edges.txt --prefix output.cluster --ncpus 1
INFO:alg_GossipMap:Running /opt/gct/submodules/PowerGraph/GossipMap --graph /tmp/tmpes78bgty/edges.txt --prefix output.cluster --ncpus 1


runing  alg_GossipMap B_alg_runtime_vs_graph_size_N4096


2020-04-24 01:07:34,679 - alg_GossipMap - INFO - Made 1 clusters in 27.559247 seconds
INFO:alg_GossipMap:Made 1 clusters in 27.559247 seconds


finished alg_GossipMap B_alg_runtime_vs_graph_size_N4096


True

In [9]:
arglist = []
badalgs={'cdc_CONGA',
 'cdc_DEMON',
 'cdc_EAGLE',
 'cdc_FastCpm',
 'cdc_GCE',
 'cdc_HDEMON',
 'cdc_LinkCommunities',
 'cdc_ParCPM',
 'igraph_community_edge_betweenness',
 'igraph_community_optimal_modularity',
 'oslom_Infomap',
 'snap_Girvan_Newman'}

datasets=[data.name]
for alg in gct.list_algorithms():
    if alg not in badalgs:
        for name in datasets:
            arglist.append((alg,name))
len(arglist)

60

In [10]:
if False:
    ret=[]
    for x in arglist:
        ret.append(run_alg_timeout_mp(x))
else:
    from joblib import Parallel, delayed
    ret=Parallel(n_jobs=6)(delayed(run_alg_timeout_mp)(x) for x in arglist)    

/opt/conda/envs/python3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [11]:
import numpy as np
np.where(not ret), len(ret)

NameError: name 'ret' is not defined

In [12]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if prefix + "_N4096" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            results[u + run] = a

len(results)

55

In [13]:
import pandas as pd
df = pd.DataFrame([(u['runname'],u['dataname'],u['timecost']) for u in results.values()], columns=['alg','datanem','timecost'])

In [14]:
set(gct.list_algorithms()).difference(df['alg'].values)


{'cdc_CONGA',
 'cdc_Connected_Iterative_Scan',
 'cdc_DEMON',
 'cdc_EAGLE',
 'cdc_FastCpm',
 'cdc_GCE',
 'cdc_HDEMON',
 'cdc_LinkCommunities',
 'cdc_MOSES',
 'cdc_ParCPM',
 'igraph_community_edge_betweenness',
 'igraph_community_optimal_modularity',
 'igraph_community_spinglass',
 'oslom_Infomap',
 'oslom_OSLOM',
 'pycabem_LabelRank',
 'snap_Girvan_Newman'}

In [15]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (df.sort_values("timecost"))

,alg,datanem,timecost
38,networkit_PLM,B_alg_runtime_vs_graph_size_N4096,0.187056
30,igraph_community_label_propagation,B_alg_runtime_vs_graph_size_N4096,0.257631
39,networkit_PLP,B_alg_runtime_vs_graph_size_N4096,0.367136
37,networkit_LPDegreeOrdered,B_alg_runtime_vs_graph_size_N4096,0.510347
54,alg_streamcom,B_alg_runtime_vs_graph_size_N4096,0.578862
48,scan_ppScanSSE,B_alg_runtime_vs_graph_size_N4096,0.664852
47,scan_ppScan,B_alg_runtime_vs_graph_size_N4096,0.724438
32,igraph_community_multilevel,B_alg_runtime_vs_graph_size_N4096,2.146816
45,scan_Scanpp,B_alg_runtime_vs_graph_size_N4096,2.262372
27,dct_seq_louvain,B_alg_runtime_vs_graph_size_N4096,3.673661
